# Introduction

In [1]:
import ipywebcam

In [2]:
w = ipywebcam.WebCamWidget()

In [3]:
w

WebCamWidget()

In [4]:
w.devicesWidget

Dropdown(description='cameras', options=(), value=None)

In [7]:
w.devices

[{'deviceId': '798221879d265b08da49754fb3f1db1670548e633067d0633ca7a71fcc980560',
  'kind': 'videoinput',
  'label': 'Chicony USB 2.0 Camera (04f2:b34c)',
  'groupId': 'c016069073d003a4211e594ae81f05849235779a4c60693b83db68ae630f5d07'}]

In [10]:
w.device